In [1]:
#Install Package

!pip install google-cloud
!pip install google-cloud-pubsub


     |████████████████████████████████| 219 kB 5.0 MB/s 
     |████████████████████████████████| 2.7 MB 42.1 MB/s 
     |████████████████████████████████| 45 kB 2.5 MB/s 
     |████████████████████████████████| 111 kB 40.3 MB/s 
     |████████████████████████████████| 596 kB 55.5 MB/s 
     |████████████████████████████████| 1.1 MB 55.1 MB/s 
  Created wheel for grpc-google-iam-v1: filename=grpc_google_iam_v1-0.12.3-py3-none-any.whl size=18515 sha256=7e78ab7c82b636d53bfbac6ce71688f479ec8d5fec43df7ce47977e77785631d
  Stored in directory: /root/.cache/pip/wheels/b9/ee/67/2e444183030cb8d31ce8b34cee34a7afdbd3ba5959ea846380
Successfully built grpc-google-iam-v1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall:

In [2]:
import os
import requests
import json
from time import sleep
from google.cloud import pubsub_v1

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'course-big-data-336218-6f8faaedbbf4.json'
# TODO(developer)
#project_id = "your-project-id"
project_id = "course-big-data-336218"
# topic_id = "your-topic-id"
topic_id = "vuelos-topic"
publisher = pubsub_v1.PublisherClient()
project_path = f"projects/{project_id}"

# AVIATION API parameters
access_key = "3107b63b4709b6dfd083a1193e013347"
limit = "5"
flight_status = "active"
arr_iata = "BCN"

aviation_url = "http://api.aviationstack.com/v1/flights" + \
            "?access_key=" + access_key + \
            "&flight_status=" + flight_status + \
            "&arr_iata=" + arr_iata
            # "&limit=" + limit  

topic_path = publisher.topic_path(project_id, topic_id)
# Main code
while True:
    # Get all flights
    response = requests.get(aviation_url)
    if response.status_code != 200:
        print("Algo ocurrió. ERROR: ", response.status_code)
        print(response.json)
        exit()

    json_text = response.text

    python_obj = json.loads(json_text)

    data_array = []
    for flight in python_obj['data']:
        data = {}

        # Only flight in the air
        if flight['live'] != None:
            data['vuelo'] = flight['flight']['iata']
            data['aerolinea'] = flight['airline']['name']
            data['fecha_vuelo'] = flight['flight_date']
            data['origen'] = flight['departure']['airport']
            data['destino'] = flight['arrival']['airport']           
            data['latitud'] = flight['live']['latitude']
            data['longitud'] = flight['live']['longitude']
            data['altitud'] = flight['live']['altitude']
            data['velocidad'] = flight['live']['speed_horizontal']
            data['timestamp'] = flight['live']['updated']
            print("Enviando información del vuelo " + data['vuelo'])
            data_array.append(data)

    print("\nEnviando array: ")
    print(data_array)

    # When you publish a message, the client returns a future.
    future = publisher.publish(topic_path, json.dumps(data_array).encode("utf-8"))
    print(future.result())

    print("\n")

    # Wait till next refresh    
    sleep(5)

Enviando información del vuelo TO3234
Enviando información del vuelo LH1138
Enviando información del vuelo VY8463
Enviando información del vuelo FR8393
Enviando información del vuelo LH1808

Enviando array: 
[{'vuelo': 'TO3234', 'aerolinea': 'Transavia France', 'fecha_vuelo': '2022-02-03', 'origen': 'Orly', 'destino': 'El Prat De Llobregat', 'latitud': 46.73, 'longitud': 2.66, 'altitud': 11887.2, 'velocidad': 825.948, 'timestamp': '2022-02-03T11:21:57+00:00'}, {'vuelo': 'LH1138', 'aerolinea': 'Lufthansa', 'fecha_vuelo': '2022-02-03', 'origen': 'Frankfurt International Airport', 'destino': 'El Prat De Llobregat', 'latitud': 44.63, 'longitud': 5.7, 'altitud': 11270, 'velocidad': 775.116, 'timestamp': '2022-02-03T21:33:51+00:00'}, {'vuelo': 'VY8463', 'aerolinea': 'Vueling', 'fecha_vuelo': '2022-02-03', 'origen': 'Lisbon Portela', 'destino': 'El Prat De Llobregat', 'latitud': 40.78, 'longitud': -3.51, 'altitud': 10599.4, 'velocidad': 888.984, 'timestamp': '2022-02-03T21:43:56+00:00'}, {'vu

KeyboardInterrupt: ignored